# Ануфриев Н.В.

Цель задания - спрогнозировать годовой доход более или менее 50 000$.

Разрешается использовать любые инструменты библиотеки scikit-learn, кроме методов глубокого обучения. Запрещается использовать другие библиотеки машинного обучения и их инструменты (имеется ввиду catboost, xgboost и подобрые).

Переменные в датасете:

- age (возраст): continuous.
- workclass (работадатель): Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt (final weight) – примерная оценка количества людей, которое представляет каждая строка данных: continuous.
- education (образование): Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status (семейное положение): Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation (сфера деятельности): Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship (родство): Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race (расса): White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex (пол): Female, Male.
- capital-gain (прирост капитала): continuous.
- capital-loss (убыток капитала): continuous.
- hours-per-week (рабочих часов в неделю): continuous.
- native-country (место рождения): United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands

### Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import category_encoders as ce

from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV, train_test_split, learning_curve, ShuffleSplit, cross_val_score, KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, Lasso, Ridge, Perceptron, LogisticRegression
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

### Загружаем датасеты

In [2]:
train_df = pd.read_csv('income_train.csv')
test_df = pd.read_csv('income_test.csv')

In [3]:
train_df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,37,Self-emp-not-inc,112497,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K
1,39,Private,278557,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1628,48,United-States,<=50K
2,33,Private,173858,Bachelors,13,Married-civ-spouse,Adm-clerical,Other-relative,Asian-Pac-Islander,Male,0,0,40,India,<=50K
3,35,?,120074,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,30,United-States,<=50K
4,21,Private,199915,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,40,United-States,<=50K


In [4]:
test_df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,43,Private,154076,Some-college,10,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,United-States
1,58,Private,236596,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
2,37,Self-emp-inc,51264,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States
3,42,Private,250121,11th,7,Married-civ-spouse,Other-service,Wife,White,Female,0,0,40,United-States
4,40,Self-emp-not-inc,89413,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1672,40,United-States


### Проверяем размерность

In [5]:
train_df.shape, test_df.shape

((26048, 15), (6513, 14))

### Проверяем информацию о данных

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26048 entries, 0 to 26047
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26048 non-null  int64 
 1   workclass       26048 non-null  object
 2   fnlwgt          26048 non-null  int64 
 3   education       26048 non-null  object
 4   education-num   26048 non-null  int64 
 5   marital-status  26048 non-null  object
 6   occupation      26048 non-null  object
 7   relationship    26048 non-null  object
 8   race            26048 non-null  object
 9   sex             26048 non-null  object
 10  capital-gain    26048 non-null  int64 
 11  capital-loss    26048 non-null  int64 
 12  hours-per-week  26048 non-null  int64 
 13  native-country  26048 non-null  object
 14  income          26048 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.0+ MB


### Смотрим статистику по всем int значениям

In [7]:
train_df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,26048.000000,2.604800e+04,26048.000000,26048.000000,26048.000000,26048.000000
mean,38.573480,1.898020e+05,10.088874,1068.759598,88.646422,40.451973
std,13.621556,1.050073e+05,2.575100,7282.271422,406.059200,12.320676
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.183090e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783190e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376340e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


### Количество уникальных значений для каждого столбца 

In [8]:
train_df.nunique()

age                  72
workclass             9
fnlwgt            18385
education            16
education-num        16
marital-status        7
occupation           15
relationship          6
race                  5
sex                   2
capital-gain        117
capital-loss         91
hours-per-week       93
native-country       42
income                2
dtype: int64

In [9]:
print('workclass', train_df.workclass.unique())
print('education', train_df.education.unique())
print('marital-status', train_df['marital-status'].unique())
print('occupation', train_df.occupation.unique())
print('relationship', train_df.relationship.unique())
print('race', train_df.race.unique())
print('sex',train_df.sex.unique())
print('native-country', train_df['native-country'].unique())
print('income', train_df.income.unique())

workclass ['Self-emp-not-inc' 'Private' '?' 'Self-emp-inc' 'Local-gov' 'Federal-gov'
 'State-gov' 'Without-pay' 'Never-worked']
education ['Masters' 'Assoc-voc' 'Bachelors' 'HS-grad' 'Some-college' 'Prof-school'
 '11th' 'Doctorate' '12th' '9th' 'Assoc-acdm' '10th' '1st-4th' 'Preschool'
 '5th-6th' '7th-8th']
marital-status ['Married-civ-spouse' 'Never-married' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Widowed' 'Married-AF-spouse']
occupation ['Exec-managerial' 'Craft-repair' 'Adm-clerical' '?' 'Other-service'
 'Tech-support' 'Sales' 'Prof-specialty' 'Machine-op-inspct'
 'Farming-fishing' 'Transport-moving' 'Handlers-cleaners'
 'Protective-serv' 'Priv-house-serv' 'Armed-Forces']
relationship ['Husband' 'Other-relative' 'Own-child' 'Not-in-family' 'Wife' 'Unmarried']
race ['White' 'Asian-Pac-Islander' 'Black' 'Other' 'Amer-Indian-Eskimo']
sex ['Male' 'Female']
native-country ['United-States' 'India' '?' 'Mexico' 'Taiwan' 'Puerto-Rico' 'Philippines'
 'Poland' 'Canada' 'Germany' 'Dom

### Ищем пропущенные данные и заменяем их

In [10]:
train_df.isin(['?']).sum()

age                  0
workclass         1501
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1506
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     468
income               0
dtype: int64

In [11]:
test_df.isin(['?']).sum()

age                 0
workclass         335
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        337
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    115
dtype: int64


### Заменяем ? на NaN

In [12]:
train_df = train_df.replace('?', np.nan)

In [13]:
train_df.isin(['?']).sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [14]:
test_df = test_df.replace('?', np.nan)

In [15]:
train_df.isin(['?']).sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [16]:
train_df.dropna(inplace=True,axis=0)
test_df.dropna(inplace=True,axis=0)

### Разделяем данные на тест/трейн

In [17]:
y = train_df.iloc[:,-1].values
X = train_df.iloc[:,:-1].values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)

In [19]:
X_train = pd.DataFrame(X_train, columns=train_df.columns[:-1])
y_train = pd.DataFrame(y_train)
X_test = pd.DataFrame(X_test, columns=train_df.columns[:-1])
y_test = pd.DataFrame(y_test)

### Инкодируем данные

In [20]:
column_trans = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), ['sex', 'race', 'workclass', 'marital-status', 'occupation']), 
    ('drop', ['fnlwgt', 'education', 'relationship', 'capital-gain', 'capital-loss', 'native-country']),
    remainder=StandardScaler())

In [21]:
X_train = column_trans.fit_transform(X_train)
X_test = column_trans.transform(X_test)
test_df = column_trans.transform(test_df)

In [22]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

### Поиск лучшей модели
### Метод К-ближайших соседей

In [23]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
score_knn = knn.score(X_test, y_test)
print('Точность К-ближайших соседей модели: ', score_knn)

Точность К-ближайших соседей модели:  0.8161825726141079


### Метод GaussianNB

In [24]:
# gaussian = GaussianNB()
# gaussian.fit(X_train, y_train)
# score_gaussian = gaussian.score(X_test,y_test)
# print('Точность Гауссовской модели: ', score_gaussian)

### Метод RFM

In [25]:
randomforest = RandomForestClassifier()
randomforest.fit(X_train, y_train)
score_randomforest = randomforest.score(X_test,y_test)
print('Точность RFM модели: ', score_randomforest)

Точность RFM модели:  0.812448132780083


### Метод логистической регрессии

In [26]:
log_reg = LogisticRegression(solver='lbfgs', max_iter=400)
log_reg.fit(X_train, y_train)
score_logreg = log_reg.score(X_test,y_test)
print('Точность логистической регрессии: ', score_logreg)

Точность логистической регрессии:  0.8265560165975103


In [27]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

cv_c = []
accuracy = []
std = []

classifiers = ['Logistic Regression', 'KNN', 'Random Forest']
models = [LogisticRegression(solver='lbfgs', max_iter=400), 
          KNeighborsClassifier(n_neighbors=9), 
          RandomForestClassifier(n_estimators=100)]
for i in models:
    model = i
    cv_result = cross_val_score(model, X_train, y_train, cv = kfold, scoring = "accuracy")
    cv_result = cv_result
    cv_c.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
    
models_df = pd.DataFrame({'CV': cv_c,'Std': std}, index=classifiers)       
models_df

,CV,Std
Logistic Regression,0.830420,0.008131
KNN,0.823987,0.006681
Random Forest,0.813613,0.006242


In [28]:
model = RandomForestClassifier() 
parameters = {'n_estimators': [150], 
              'max_features': ['log2', 'sqrt', 'auto'], 
              'max_depth': [30, 50, 70, 90], 
              'min_samples_split': [5, 7, 9, 11]} 
rfc = GridSearchCV(model, parameters, scoring = 'accuracy', n_jobs=-1, cv=5) 
rfc = rfc.fit(X_train,y_train) 

print('Best params: ', rfc.best_params_)
print('\nBest score: ', rfc.best_score_)
print('\nBest estimator: ', rfc.best_estimator_)

Best params:  {'max_depth': 50, 'max_features': 'auto', 'min_samples_split': 11, 'n_estimators': 150}

Best score:  0.8335321509490827

Best estimator:  RandomForestClassifier(max_depth=50, min_samples_split=11, n_estimators=150)


In [30]:
pipeline_final = make_pipeline(RandomForestClassifier(max_depth=50, max_features='log2', min_samples_split=11, 
                                                      n_estimators=150))
pipeline_final.fit(X_train, y_train)

y_predict = pipeline_final.predict(test_df)
y_predict

array(['<=50K', '>50K', '>50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)